## Imports

In [1]:
import os

from mmaction.datasets import build_dataset, build_dataloader
from mmcv import Config

/home/sadat/miniconda3/envs/scrambmix/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sadat/miniconda3/envs/scrambmix/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


## Loading batches

In [2]:
cfg = Config.fromfile('./baseline.py')

In [3]:
os.chdir('../../..')

In [4]:
cfg.data.train

{'type': 'RawframeDataset',
 'ann_file': 'data/hmdb51/annotation_train.txt',
 'data_prefix': 'data/hmdb51/rawframes',
 'pipeline': [{'type': 'SampleFrames',
   'clip_len': 32,
   'frame_interval': 2,
   'num_clips': 1},
  {'type': 'RawFrameDecode'},
  {'type': 'Resize', 'scale': (-1, 256)},
  {'type': 'RandomResizedCrop'},
  {'type': 'Resize', 'scale': (224, 224), 'keep_ratio': False},
  {'type': 'Flip', 'flip_ratio': 0.5},
  {'type': 'Normalize',
   'mean': [123.675, 116.28, 103.53],
   'std': [58.395, 57.12, 57.375],
   'to_bgr': False},
  {'type': 'FormatShape', 'input_format': 'NCTHW'},
  {'type': 'Collect', 'keys': ['imgs', 'label'], 'meta_keys': []},
  {'type': 'ToTensor', 'keys': ['imgs', 'label']}]}

In [5]:
train_dataset = build_dataset(cfg=cfg.data.train)
train_loader = build_dataloader(
        train_dataset,
        videos_per_gpu=10,
        workers_per_gpu=4,
        persistent_workers=False,
        num_gpus=1,
        dist=False)

val_dataset = build_dataset(cfg=cfg.data.val)
val_loader = build_dataloader(
        val_dataset,
        videos_per_gpu=1,
        workers_per_gpu=4,
        persistent_workers=False,
        num_gpus=1,
        dist=False)

In [6]:
iter_ = iter(train_loader)
result = next(iter_)

In [7]:
result.keys()

dict_keys(['imgs', 'label'])

## Model

In [8]:
from mmaction.models import build_model

In [9]:
# Backbone hyperparameters
cfg.model.backbone.with_pool2 = False
cfg.model.backbone.pretrained = 'https://download.openmmlab.com/mmaction/recognition/csn/ircsn_from_scratch_r50_ig65m_20210617-ce545a37.pth'
cfg.model.backbone.bottleneck_mode = 'ir'
cfg.model.backbone.norm_eval = True
cfg.model.backbone.bn_frozen = True

# Head hyperparameters
cfg.model.cls_head.spatial_type='avg'
cfg.model.cls_head.dropout_ratio=0.5

cfg.model

{'type': 'Recognizer3D',
 'backbone': {'type': 'ResNet3dCSN',
  'pretrained2d': False,
  'pretrained': 'https://download.openmmlab.com/mmaction/recognition/csn/ircsn_from_scratch_r50_ig65m_20210617-ce545a37.pth',
  'depth': 50,
  'with_pool2': False,
  'bottleneck_mode': 'ir',
  'norm_eval': True,
  'zero_init_residual': False,
  'bn_frozen': True},
 'cls_head': {'type': 'I3DHead',
  'num_classes': 52,
  'in_channels': 2048,
  'spatial_type': 'avg',
  'dropout_ratio': 0.5,
  'init_std': 0.01},
 'train_cfg': None,
 'test_cfg': {'average_clips': 'prob', 'max_testing_views': 10}}

In [10]:
model = build_model(
        cfg.model, train_cfg=None, test_cfg=cfg.get('test_cfg'))

2024-11-04 18:06:25,682 - mmaction - INFO - load model from: https://download.openmmlab.com/mmaction/recognition/csn/ircsn_from_scratch_r50_ig65m_20210617-ce545a37.pth
2024-11-04 18:06:25,683 - mmaction - INFO - load checkpoint from http path: https://download.openmmlab.com/mmaction/recognition/csn/ircsn_from_scratch_r50_ig65m_20210617-ce545a37.pth


## Learning Hyperparameters

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(
    model.parameters(),
    lr=0.000125,
    momentum=0.9,
    weight_decay=0.0001
)

# Gradient clipping configuration
max_norm = 40
norm_type = 2

# Learning rate scheduler
scheduler = MultiStepLR(optimizer, milestones=[70, 140], gamma=0.1)

# Warmup settings
warmup_epochs = 16
warmup_start_lr = 0.000125 * 0.1  # warmup_ratio=0.1
initial_lr = 0.000125

# Training configuration
total_epochs = 50
eval_interval = 5

## Train Loop

In [12]:
import wandb
from tqdm import tqdm
import torch

# Initialize Weights and Biases
wandb.init(project='hmdb51-tuning', entity='cares', group='baseline')  # Replace with your project and entity names

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Initialize log file
log_file = 'training_log.txt'  # Specify the log file name
with open(log_file, 'w') as f:  # Open the log file for writing
    for epoch in range(total_epochs):
        # Warmup phase
        if epoch < warmup_epochs:
            warmup_lr = warmup_start_lr + (initial_lr - warmup_start_lr) * (epoch / warmup_epochs)
            for param_group in optimizer.param_groups:
                param_group['lr'] = warmup_lr
        else:
            scheduler.step()  # Step the scheduler after warmup

        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        # Training step
        for i, data in enumerate(tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{total_epochs}")):
            inputs, labels = data['imgs'].to(device), data['label'].to(device)

            optimizer.zero_grad()
            results = model.forward(inputs, labels, return_loss=True)
            loss = results['loss_cls']
            train_accuracy = results['top1_acc']  

            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm, norm_type)
            
            optimizer.step()

            running_loss += loss.item()
            correct += (results['top1_acc'] * inputs.size(0))  # Assuming top1_acc is in percentage
            total += inputs.size(0)

        train_loss = running_loss / len(train_loader)
        train_accuracy = correct / total

        # Log training loss and accuracy
        log_line = f"Epoch [{epoch+1}/{total_epochs}], Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}\n"
        f.write(log_line)
        print(log_line.strip())  # Print to console for real-time feedback

        # Log training metrics to WandB
        wandb.log({
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'train_accuracy': train_accuracy
        })

        # Validation phase
        if (epoch + 1) % eval_interval == 0:
            model.eval()
            val_running_loss = 0.0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for val_data in tqdm(val_loader, desc=f"Validating Epoch {epoch+1}/{total_epochs}"):
                    val_inputs, val_labels = val_data['imgs'].to(device), val_data['label'].to(device)
                    
                    val_results = model.forward(val_inputs, val_labels, return_loss=True)
                    val_loss = val_results['loss_cls']
                    val_accuracy = val_results['top1_acc']

                    val_running_loss += val_loss.item()
                    val_correct += (val_accuracy * val_inputs.size(0))
                    val_total += val_inputs.size(0)

            val_loss = val_running_loss / len(val_loader)
            val_accuracy = val_correct / val_total

            # Log validation loss and accuracy
            val_log_line = f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}\n"
            f.write(val_log_line)
            print(val_log_line.strip())  # Print to console for real-time feedback

            # Log validation metrics to WandB
            wandb.log({
                'val_loss': val_loss,
                'val_accuracy': val_accuracy
            })

print("Training complete.")
wandb.finish()  # Finish the WandB run


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sttaseen (cares). Use `wandb login --relogin` to force relogin


Training Epoch 1/50: 100%|██████████| 357/357 [02:13<00:00,  2.67it/s]


Epoch [1/50], Training Loss: 3.9332, Training Accuracy: 0.0347


Training Epoch 2/50: 100%|██████████| 357/357 [02:09<00:00,  2.76it/s]


Epoch [2/50], Training Loss: 3.7461, Training Accuracy: 0.0927


...

Epoch [37/50], Training Loss: 0.5345, Training Accuracy: 0.8538


Training Epoch 38/50: 100%|██████████| 357/357 [01:55<00:00,  3.10it/s]


Epoch [38/50], Training Loss: 0.4989, Training Accuracy: 0.8597


Training Epoch 39/50:  86%|████████▋ | 308/357 [01:42<00:16,  3.00it/s]


KeyboardInterrupt: 